In [14]:
from simulator import Simulator as sim
import numpy as np

sim0 = sim(
    dt=0.001,
    prediction_horizon=200,
    simulation_time=5,
    surface_limits=((-2, 2), (-2, 2)),
    surface_origin=np.array([0.0, 0.0, 0.0]),
    surface_orientation_rpy=np.array([0.0, 0.0, 0.0]),
    qdot_0=np.array([2,2,2,2,2,2]),
    q_0=np.array([np.pi/3, -np.pi/3, np.pi/4, -np.pi/2, -np.pi/2, 0.0]),
    wcv=np.array([5,10,15,20,25,35]),
    scene=True
)
sim0.run()
sim0_results = sim0.get_results()

URDF successfully loaded: /Users/bb/Desktop/robotic-mpc/ur_description/urdf/ur5.urdf
nq = 6, ngeoms(vis) = 7, ngeoms(col) = 7
Pinocchio model and data successfully created.
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/
rm -f libacados_ocp_solver_six_dof_robot_model.dylib
rm -f six_dof_robot_model_cost/six_dof_robot_model_cost_y_0_fun.o six_dof_robot_model_cost/six_dof_robot_model_cost_y_0_fun_jac_ut_xt.o six_dof_robot_model_cost/six_dof_robot_model_cost_y_0_hess.o six_dof_robot_model_cost/six_dof_robot_model_cost_y_fun.o six_dof_robot_model_cost/six_dof_robot_model_cost_y_fun_jac_ut_xt.o six_dof_robot_model_cost/six_dof_robot_model_cost_y_hess.o acados_solver_six_dof_robot_model.o
cc -fPIC -std=c99   -O2 -I/Users/bb/Desktop/robotic-mpc/acados/include -I/Users/bb/Desktop/robotic-mpc/acados/include/acados -I/Users/bb/Desktop/robotic-mpc/acados/include/blasfeo/include -I/Users/bb/Desktop/robotic-mpc/acados/include/hpipm/include  -c -o six_dof_rob

ld: warning: search path '-l' not found


Integration time: 0.0002760887145996094 s
Running time: 0.0 s of 5 s
Integration time: 0.00010085105895996094 s
Integration time: 9.608268737792969e-05 s
Integration time: 0.00011873245239257812 s
Integration time: 9.489059448242188e-05 s
Integration time: 9.703636169433594e-05 s
Integration time: 0.00010395050048828125 s
Integration time: 0.0001010894775390625 s
Integration time: 9.083747863769531e-05 s
Integration time: 9.894371032714844e-05 s
Integration time: 9.489059448242188e-05 s
Running time: 0.01 s of 5 s
Integration time: 9.799003601074219e-05 s
Integration time: 7.200241088867188e-05 s
Integration time: 6.198883056640625e-05 s
Integration time: 5.1975250244140625e-05 s
Integration time: 9.894371032714844e-05 s
Integration time: 6.103515625e-05 s
Integration time: 4.887580871582031e-05 s
Integration time: 9.608268737792969e-05 s
Integration time: 0.00010085105895996094 s
Integration time: 8.416175842285156e-05 s
Running time: 0.02 s of 5 s
Integration time: 0.0001142024993896

In [ ]:
from plotter import Plotter
from matplotlib import pyplot as plt

plotter = Plotter()
t = sim0_results['time']
q = sim0_results['q']

#Joint Angles Plot
fig_joints = plotter.joint_angles(t, q, title="Joint Angles")

# #e1 error plot
# ee_xyz = sim0_results["ee_pose"][:3]

# R_ee_t = np.array([
#     [1.0,  0.0,  0.0],
#     [0.0, 1.0,  0.0],
#     [0.0,  0.0, 1.0],
# ])
# ee_task = R_ee_t @ ee_xyz + np.array([0.0, 0.0, 1.0])[:, np.newaxis]

# e1 = np.zeros(ee_task.shape[1])
# z_surface = np.zeros(ee_task.shape[1])

# for i in range(ee_task.shape[1]):
#     e1[i] = sim0.surface.get_point_on_surface(ee_task[0, i], ee_task[1, i]) - ee_task[2, i]
#     z_surface[i] = sim0.surface.get_point_on_surface(ee_task[0, i], ee_task[1, i])
# fig_e1 = plotter.generic_plot(t, e1, z_surface, xlabel="$t$ [s]", ylabel="$e1$ [m]", title="E1 Error", labels=["$e1$", "$z_{surface}$"])

#Acados
stats = sim0.mpc.solver.get_stats('statistics')

#SQP Iteration Plot
sqp_iter = np.arange(1,sim0.mpc.solver.get_stats('sqp_iter') + 1)
fig_cost_itr = plotter.generic_plot(t, stats[3,:], xlabel="$i$ [s]", ylabel="Cost Function Value", title="Cost vs. SQP Iteration", labels=["Cost"])
print(sim0.mpc.solver.get_stats("statistics")[:,0].shape)
print(sim0.mpc.solver.get_stats("residuals").shape)

#Timing Performance Plot
fig_timing = plotter.generic_plot(
    t, 
    sim0.mpc_time, 
    sim0.integration_time, 
    xlabel="$t$ [s]", 
    ylabel="$t$ [s]", 
    title="Timing Performance", 
    labels=["MPC", "Integration"])

# Plot cost vs SQP iteration
#Show Plots
plotter.show(fig_joints)
plotter.show(fig_timing)
plotter.show(fig_cost_itr)
plotter.show(fig_e1)

(8,)
(4,)
